In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import src.load.load_image as li
import src.base.resize_image as ri
import src.base.connect_to_database as ctd
from shapely import wkt

# set the paths
path_dem = "/home/fdahle/Desktop/agi_test/output2/dem.tif"
path_ortho = "/home/fdahle/Desktop/agi_test/output2/ortho.tif"

# set image ids
image_ids = ["CA184832V0146", "CA184832V0147", "CA184832V0148", "CA184832V0149", "CA184832V0150"]

# load data
dem = li.load_image(path_dem)
ortho = li.load_image(path_ortho)

# resize ortho to same size as dem
ortho = ri.resize_image(ortho, dem.shape)

# load footprints
conn = ctd.establish_connection()
image_ids_str = "','".join(image_ids)
sql_string = (f"SELECT st_astext(footprint_exact) FROM images_georef "
              f" WHERE image_id IN ('{image_ids_str}')")
data = ctd.execute_sql(sql_string, conn)
footprints = data['st_astext'].tolist()
footprints = [wkt.loads(footprint) for footprint in footprints]

In [17]:
import src.sfm_agi.snippets.create_gcps as cg

print(dem.shape, ortho.shape)

output_path = "/home/fdahle/Desktop/agi_test/markers.csv"

cg.create_gcps(dem, ortho, footprints, output_path)

(2243, 4073) (2243, 4073)
(300, 2)
       GCP             x              y
0    gcp_1 -1.996584e+06  731511.935034
1    gcp_2 -1.996359e+06  731348.054765
2    gcp_3 -1.994796e+06  731172.460800
3    gcp_4 -1.992668e+06  730652.427212
4    gcp_5 -1.992384e+06  730245.869099
..     ...           ...            ...
86  gcp_87 -1.992257e+06  723597.030979
87  gcp_88 -1.990835e+06  723496.364188
88  gcp_89 -1.990082e+06  723238.485336
89  gcp_90 -1.988088e+06  722508.301346
90  gcp_91 -1.987623e+06  722675.173074

[91 rows x 3 columns]


/home/fdahle/Documents/Github/Antarctic_TMA/src/dem/find_peaks_in_DEM.py:42: PeakPropertyWarning: some peaks have a prominence of 0
  peak_prominences_vals, left_bases, right_bases = peak_prominences(dem.flatten(), peak_indices)
